In [1]:
from pyspark import SparkContext
# initialize a new Spark Context to use for the execution of the script
sc = SparkContext(appName="MY-APP-NAME", master="local[*]")

In [8]:
# EXERCISE N.1
# Compute the numer of words appearing in both commedies and tragedies and the number of words which are not 
# present in both the texts

text1 = sc.textFile("hdfs://kddrtserver11.isti.cnr.it:9000/user/hpsa00/comedies.txt")
text2 = sc.textFile("hdfs://kddrtserver11.isti.cnr.it:9000/user/hpsa00/tragedies.txt")
words1 = text1.flatMap(lambda x: x.split())
words2 = text2.flatMap(lambda x: x.split())
both = words1.intersection(words2)

print(both.count())
print(words1.count()+words2.count()-both.count())

In [5]:
# EXERCISE N.2
# Read the file commedies and count the number of rows containing the word "skill"

text = sc.textFile("hdfs://kddrtserver11.isti.cnr.it:9000/user/hpsa00/comm2.txt")
pair_rdd = text.map(lambda x: x.lower().split()).filter(lambda x: "skill" in x)
print (pair_rdd.count())

[["'had'!", 'how', 'sad', 'a', 'passage', "'tis!--whose", 'skill', 'was'], ['skill', 'infinite', 'or', 'monstrous', 'desperate.'], ['is', 'dearest', 'to', 'me.', 'i', 'have', 'no', 'skill', 'in', 'sense'], ['skill', 'in', 'grass.'], ['here', 'stand', 'i', 'lady,', 'dart', 'thy', 'skill', 'at', 'me;'], ['truly,', 'my', 'ancient', 'skill', 'beguiles', 'me;', 'but,', 'in', 'the'], ['page', 'i', 'have', 'heard', 'the', 'frenchman', 'hath', 'good', 'skill', 'in'], ['this', 'philoten', 'contends', 'in', 'skill'], ['my', 'utmost', 'skill', 'in', 'his', 'recovery,', 'provided'], ['youth,', 'strength,', 'skill', 'and', 'wrath', 'can', 'furnish', 'man', 'withal.'], ['as', 'little', 'skill', 'to', 'fear', 'as', 'i', 'have', 'purpose']]
11


In [49]:
# EXERCISE N.3
# Read the file commedies and count the number of words having the same second letter.

text = sc.textFile("hdfs://kddrtserver11.isti.cnr.it:9000/user/hpsa00/comm2.txt")
pair_rdd = text.flatMap(lambda x: x.split()).filter(lambda x: len(x)>2).map(lambda x: (x[1].upper(), 1)).reduceByKey(lambda x,y:x+y)
print (pair_rdd.collect())

In [7]:
# EXERCISE N.4
# Create a file with the following list of class,member pairs:
# 1,2;1,2;1,3;1,4;1,5;2,1;2,4;2,4;2,7;2,9;3,1;3,2;3,3;3,6;3,1
# Read the file and write the code to obtain the list of elements for each class

text_rdd = sc.textFile("ex1.txt")
pairs_rdd = text_rdd.flatMap(lambda x: x.split(";")).map(lambda x: x.split(",")).reduceByKey(lambda x,y: x+","+y)

print (pairs_rdd.collect())

[('1', '2,2,3,4,5'), ('2', '1,4,4,7,9'), ('3', '1,2,3,6,1')]


In [117]:
# EXERCISE N.5
# Generate 1000 random points (x,y) and 20 seed points(a,b). Assign the points to the closer seed and than 
# count how many points are assigned to each seed.

from random import random
import numpy as np

def closer(elem,listS):
    min = 1000000
    min_i = -1
    for e in listS:
        elem=np.array(elem)
        seed=np.array(e[1])
        p=np.sum((elem-seed)**2)
        d=np.sqrt(p)
        if min>d: 
            min = d
            min_i = e[0]
    return min_i

points = []
for _ in range(1000):
    value_x = random()
    value_y = random()
    points.append((value_x,value_y))
    
seeds = []
for i in range(20):
    value_x = random()
    value_y = random()
    seeds.append((i,(value_x,value_y)))
    
points_rdd = sc.parallelize(points)
distances_rdd = points_rdd.map(lambda x: closer(x, seeds)).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y).sortByKey()

print(distances_rdd.collect())

[(0, 28), (1, 58), (2, 67), (3, 44), (4, 10), (5, 29), (6, 42), (7, 39), (8, 31), (9, 81), (10, 54), (11, 46), (12, 88), (13, 30), (14, 38), (15, 51), (16, 62), (17, 80), (18, 89), (19, 33)]
